# Comandos selenium para acessar o site

In [1]:
from dotenv import load_dotenv
import os 
load_dotenv()

CPF_TALUDE = str(os.getenv("CPF_TALUDE"))
COD_UNIODONTO = str(os.getenv("COD_UNIODONTO"))
PASSWORD = str(os.getenv("PASSWORD"))

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

opts = Options()
# opts.add_argument("--headless=new")  # remova se quiser ver a janela
opts.add_argument("--window-size=1366,768")
opts.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=opts)  # Selenium Manager resolve o driver


In [3]:
driver.get("https://www.uniodontosc.coop.br/cooperados/index.php")

In [4]:
from selenium.webdriver.common.by import By
from time import sleep

In [5]:
sleep(3)

In [6]:
cpf_bar = driver.find_element(By.ID, 'campoCpf')
sleep(1)
cod_bar = driver.find_element(By.ID, 'campoCodigo')
sleep(1)
paswd_bar = driver.find_element(By.ID, 'campoSenha')

In [7]:
for numero in CPF_TALUDE:
    sleep(0.5)
    cpf_bar.send_keys(numero)


In [8]:
cod_bar.send_keys(COD_UNIODONTO)

In [9]:
paswd_bar.send_keys(PASSWORD)

In [10]:
sleep(3)

In [11]:
login_button = driver.find_element(By.XPATH, '/html/body/div[2]/section/div[3]/div/div/div/div/form/div/div[4]/div/div/button')

In [12]:
login_button.click()

In [13]:
sleep(1.5)

In [14]:
close_add_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/a')

In [15]:
close_add_button.click()

In [16]:
sleep(1.5)

In [17]:
production_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/div[4]/nav/ul/li[6]/a')

In [18]:
production_button.click()

In [19]:
sleep(0.5)

In [20]:
lote_generation = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/div[4]/nav/ul/li[6]/ul/li[1]/a')

In [21]:
lote_generation.click()

In [22]:
sleep(2)

In [23]:
empresa_lote = driver.find_element(By.ID, 'empresaLote')

In [24]:
empresa_lote.click()

In [25]:
all_option = driver.find_element(By.XPATH, '/html/body/form/div/div/table/tbody/tr/td/table/tbody/tr[2]/td[3]/select/option[12]')

In [26]:
all_option.click()

In [27]:
month_year = driver.find_element(By.ID, 'mesAno')

In [28]:
from datetime import datetime

agora = datetime.now()
mes_ano = f"{agora.month:02d}/{agora.year}".replace("/", "")  # zero à esquerda no mês
mes_ano = "082025" #TESTE COM A JULIA

In [29]:
month_year.clear()
month_year.click()
sleep(1.5)
for n in mes_ano:
    sleep(1)
    month_year.send_keys(int(n))

In [30]:
search = driver.find_element(By.ID, 'pesquisarGuia')

In [31]:
search.click()

# Logica para achar o nome e o valor certo

In [32]:
file_path = "/Users/enzobarbi/Development/github/guias-uniodonto/fotos/2025-10-05_15-39-06 - Julia_Moraes_De_Souza_Faria - R$ 799,70 - GTO.jpg"
file_name = os.path.basename(file_path)  # Extrair nome do arquivo do caminho

In [33]:
data = file_name.split(" - ")
data_name = data[1].replace("_", " ")
data_price = data[2].split(" ")[1]
data_anexo = data[3].split(".")[0]

NOME = data_name
PRECO = data_price
ANEXO = data_anexo

In [34]:
print(NOME)
print(PRECO)
print(ANEXO)

Julia Moraes De Souza Faria
799,70
GTO


In [35]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:

wait = WebDriverWait(driver, 20)

# Espera a tabela existir
wait.until(EC.presence_of_element_located((By.ID, "tabelaListagem")))

# Localiza a linha que atende às duas condições:
# - td[4] contém um <a> com o texto do nome (normalizado)
# - existe na mesma linha um <td> cujo texto seja exatamente o preço
row_xpath = (
    "//table[@id='tabelaListagem']//tr["
    "  td[4]//a[normalize-space()=$nome] and "
    "  .//td[normalize-space()=$preco]"
    "]"
)

# Selenium não tem binding nativo para variáveis no XPath, então interpolamos:
row = driver.find_element(
    By.XPATH,
    row_xpath.replace("$nome", f"'{NOME}'").replace("$preco", f"'{PRECO}'")
)

# Agora pegue o link da coluna 4 e clique
link = row.find_element(By.XPATH, "td[4]//a")
driver.execute_script("arguments[0].scrollIntoView({block:'center'});", link)

try:
    wait.until(EC.element_to_be_clickable(link)).click()
except Exception:
    # se overlay/interceptação atrapalhar, força via JS
    driver.execute_script("arguments[0].click();", link)


In [37]:
sleep(3)

In [38]:
rx_button = driver.find_element(By.ID, "AnexarRx1")
gto_button = driver.find_element(By.ID, "AnexarRx2")

In [39]:
if ANEXO == "GTO":
    gto_button.click()
else:
    print("Implementar o RX")

In [40]:
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 20)

# acha o iframe do Thickbox
iframe = wait.until(EC.presence_of_element_located((
    By.CSS_SELECTOR, "iframe#TB_iframeContent, iframe[src*='imagens_lote_guias.php']"
)))

src = iframe.get_attribute("src")  # pode vir relativo
src_abs = urljoin(driver.current_url, src)
print("SRC do iframe:", src_abs)

# navega direto na URL do iframe (fora de iframe, tela cheia)
driver.get(src_abs)


SRC do iframe: https://www.sisoweb.coop.br/web/cooperados/imagens.lote.guias.php?height=550&width=900&modal=true&controle=5084960&lote=0&baixada=f&liberada=f&menu=2&


In [41]:
codigo = src_abs.split("=")[4].split("&")[0]

In [42]:
codigo

'5084960'

In [43]:
phpsessid = driver.get_cookie('PHPSESSID')['value']
print("PHPSESSID:", phpsessid)

PHPSESSID: 33aa0lpj8sh7734co3pf0sl8ue


In [44]:
import requests
import os

# Determinar content-type baseado na extensão
file_ext = os.path.splitext(file_name)[1].lower()
content_types = {
    '.jpg': 'image/jpeg',
    '.jpeg': 'image/jpeg', 
    '.png': 'image/png',
    '.pdf': 'application/pdf',
    '.doc': 'application/msword',
    '.docx': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
}
content_type = content_types.get(file_ext, 'application/octet-stream')

# URL e configurações
url = "https://www.sisoweb.coop.br/web/cooperados/upload.process.imagens.php"

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'referer': f'https://www.sisoweb.coop.br/web/cooperados/imagens.lote.guias.php?height=550&width=900&modal=true&controle={codigo}&lote=0&baixada=f&liberada=f&menu=2&'
}

cookies = {
    'PHPSESSID': phpsessid, 
    'loginCooperados': '1'
}

data = {
    'controle': str(codigo),
    'baixada': 'f',
    'liberada': 'f',
    'pendente': 'f'
}
    
# FAZER UPLOAD
with open(file_path, 'rb') as f:
    files = {
        'files[]': (file_name, f, content_type)
    }
    
    response = requests.post(url, data=data, files=files, headers=headers, cookies=cookies, timeout=120)
    

In [45]:
sleep(3)

# ENVIAR FORM

In [46]:
form = driver.find_element(By.ID, 'frmLista')

In [47]:
procedimento = form.find_element(By.ID, 'procedimento3920063')
procedimento.click()
sleep(1)
gtoprocedimento = form.find_element(By.XPATH, '/html/body/div/div/div/div[4]/div[2]/form/div[2]/table/tbody/tr[2]/td[2]/select/option[2]')
gtoprocedimento.click()

In [48]:
# concluir = driver.find_element(By.ID, 'btnEnviarArquivoImagem')
# concluir.click()

In [ ]:
sleep(3)

In [49]:
driver.get("https://www.sisoweb.coop.br/web/cooperados/gerar.lote.php#")

Repetir a parte que digita a data e procura a pessoa com base do nome

* Looping para os arquivos, coleta de nome 